# IALS Recommender

In [ ]:
def init_kaggle(run_on_kaggle, secret_name):
    if run_on_kaggle:
        from kaggle_secrets import UserSecretsClient
        user_secrets = UserSecretsClient()
        personal_token = user_secrets.get_secret(secret_name)
        
        import subprocess
        subprocess.run(["git", "clone", "https://" + personal_token + "@github.com/alecontuIT/rec_sys_challenge2022.git"])
        
        import os
        os.chdir("./rec_sys_challenge2022")
        
        subprocess.run(["pip", "install", "-r", "requirements.txt"])
        return True
    
    else:
        return False

In [ ]:
dataset_version = "interactions-all-ones"
kaggle = init_kaggle(False, "recsys_git_token")

In [ ]:
import utils
from recmodels import IALSRec
from scipy.stats import loguniform
from Evaluation.Evaluator import EvaluatorHoldout
from skopt.space import Real, Integer, Categorical
from HyperparameterTuning.SearchAbstractClass import SearchInputRecommenderArgs

In [ ]:
recommender_class = IALSRec

In [ ]:
URM_all, URM_train, URM_val, ICM = utils.get_data_global_sample(dataset_version=dataset_version, 
                                                                train_percentage=0.7,
                                                                setSeed=True)

In [ ]:
evaluator_validation = EvaluatorHoldout(URM_val, cutoff_list=[10])

## Hyperparameter Search

In [ ]:
hyperparameters_range_dictionary = {
    "epochs": Categorical([300]),
    "num_factors" : Integer(1, 200),
    "confidence_scaling" : Categorical(["linear", "log"]),
    "alpha": Real(low = 1e-3, high = 50.0, prior = 'log-uniform'),
    "epsilon": Real(low = 1e-3, high = 10.0, prior = 'log-uniform'),
    "reg": Real(low = 1e-5, high = 1e-2, prior = 'log-uniform')
}

In [ ]:
earlystopping_keywargs = {
    "validation_every_n": 5,
    "stop_on_validation": True,
    "evaluator_object": evaluator_validation,
    "lower_validations_allowed": 5,
    "validation_metric": "MAP"
}

In [ ]:
recommender_input_args = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS = [URM_train],
    CONSTRUCTOR_KEYWORD_ARGS = {},
    FIT_POSITIONAL_ARGS = [],
    FIT_KEYWORD_ARGS = {},
    EARLYSTOPPING_KEYWORD_ARGS = earlystopping_keywargs,
)

In [ ]:
utils.bayesian_search(
    recommender_class, 
    recommender_input_args, 
    hyperparameters_range_dictionary, 
    evaluator_validation,
    dataset_version=dataset_version,
    n_cases=70,
    perc_random_starts=0.3,
    resume_from_saved=True
)

## Best Model

In [ ]:
from HyperparameterTuning.SearchSingleCase import SearchSingleCase
output_folder_path = utils.get_hyperparams_search_output_folder(recommender_class, dataset_version=dataset_version)
   
fit_hyperparams = {"epochs": 250, "num_factors": 59, "confidence_scaling": "linear", "alpha": 5.8267337801306525, "epsilon": 0.818204727509452, "reg": 3.6211485102456735e-05}

recommender_input_args = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS = [URM_train],    
    CONSTRUCTOR_KEYWORD_ARGS = {},
    FIT_POSITIONAL_ARGS = [],
    FIT_KEYWORD_ARGS = {},
    EARLYSTOPPING_KEYWORD_ARGS = earlystopping_keywargs, 
)
urm = recommender_input_args.CONSTRUCTOR_POSITIONAL_ARGS[0]
hyperparameterSearch = SearchSingleCase(recommender_class, evaluator_validation=evaluator_validation)
hyperparameterSearch.search(recommender_input_args,
                                   fit_hyperparameters_values=fit_hyperparams,
                                   metric_to_optimize = "MAP",
                                   cutoff_to_optimize = 10,
                                   output_folder_path = output_folder_path,
                                   output_file_name_root = recommender_class.RECOMMENDER_NAME,
                                   resume_from_saved = False,
                                   save_model = "best"
                                   )

In [ ]:
recommender = utils.fit_best_recommender(recommender_class, URM_all, dataset_version)
#utils.submission(recommender, dataset_version)